# Yelp and FourSquare API Data Extraction of Chinese Restaurant in Calgary

## Part 1 Using Yelp API for data extraction and analysis

### step 1- Yelp API Data Extraction

In [58]:
# import libraries ( found yelpapi python library) 
import requests as re
import os
import numpy as np 
import pandas as pd
from yelpapi import YelpAPI

In [59]:
# set up enviromental variables for API_key and API_secret

Yelp_ID=os.environ['Yelp_Client_ID'] # not required
api_key=os.environ['Yelp_Client_Key']
yelp_api = YelpAPI(api_key)

In [60]:
# set up API query params location as 'Calgary chinese restaunt with limit of 50 record
search_location='Calgary, Canada'
search_term='chinese restaurant'
search_limit=50

In [61]:
# check the API response result- #response is a dictionary 
response = yelp_api.search_query(term = search_term,
                                 location =search_location,
                                 limit = search_limit)
response.keys()

dict_keys(['businesses', 'total', 'region'])

In [69]:
# further explore the data layer information
response['businesses'][0]

{'id': '43V5SUyC04za5ITSvRndog',
 'alias': 'great-taste-chinese-restaurant-calgary',
 'name': 'Great Taste Chinese Restaurant',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/qTNyQRDOODEFM9640GObRA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/great-taste-chinese-restaurant-calgary?adjust_creative=MF4ZCJdyS3f3O1jkl2e3AQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=MF4ZCJdyS3f3O1jkl2e3AQ',
 'review_count': 82,
 'categories': [{'alias': 'chinese', 'title': 'Chinese'}],
 'rating': 4.0,
 'coordinates': {'latitude': 51.05102729247, 'longitude': -114.061279112366},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '123 2 Avenue SE',
  'address2': '',
  'address3': '',
  'city': 'Calgary',
  'zip_code': 'T2G 0B2',
  'country': 'CA',
  'state': 'AB',
  'display_address': ['123 2 Avenue SE', 'Calgary, AB T2G 0B2', 'Canada']},
 'phone': '+14032659880',
 'display_phone': '+1 403-265-9880',
 'distance': 1453.6428453331075}

In [67]:
# using Json_Normalize to get the pandas dataframe

from pandas.io.json import json_normalize
yelp=json_normalize(response, record_path='businesses')  
yelp.head(1)

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,great-taste-chinese-restaurant-calgary,"[{'alias': 'chinese', 'title': 'Chinese'}]","{'latitude': 51.05102729247, 'longitude': -114...",+1 403-265-9880,1453.642845,43V5SUyC04za5ITSvRndog,https://s3-media3.fl.yelpcdn.com/bphoto/qTNyQR...,False,"{'address1': '123 2 Avenue SE', 'address2': ''...",Great Taste Chinese Restaurant,+14032659880,$$,4.0,82,[],https://www.yelp.com/biz/great-taste-chinese-r...


In [68]:
df_yelp=yelp[['name','review_count','rating']]
yelptop10=df_yelp.sort_values(by='review_count',ascending=False).head(10)
yelptop10

,name,review_count,rating
45,T.Pot China Bistro,112,3.5
7,Silver Dragon Restaurant,111,3.5
0,Great Taste Chinese Restaurant,82,4.0
19,Sun's BBQ Restaurant,79,3.5
5,Calgary Court Restaurant,74,4.0
22,U & Me Restaurant,73,3.5
49,Ginger Beef Bistro House,69,3.5
20,Bill’s Peking House,65,3.5
2,Szechuan Restaurant,64,4.0
33,Forbidden City Dim Sum & Seafood Restaurant,63,3.5


### step 2 Yelp API to SQL

In [69]:
## create SQLite Database p
import sqlite3 as sqlite
conn=sqlite.connect('p.db')
c=conn.cursor()
#c.execute('CREATE TABLE ccr (name text, review_count integer, rating float)')
#conn.commit()
 

In [71]:
#Get from pandas to SQL
df_yelp.to_sql('ccr', conn, if_exists='replace', index=False)

In [29]:
#SQL Query Result
c.execute('''Select * from ccr order by review_count DESC limit 10''')
for row in c.fetchall():
    print(row)

('T.Pot China Bistro', 112, 3.5)
('Silver Dragon Restaurant', 111, 3.5)
('Great Taste Chinese Restaurant', 82, 4.0)
("Sun's BBQ Restaurant", 79, 3.5)
('Calgary Court Restaurant', 74, 4.0)
('U & Me Restaurant', 73, 3.5)
('Ginger Beef Bistro House', 69, 3.5)
('Bill’s Peking House', 65, 3.5)
('Szechuan Restaurant', 64, 4.0)
('Forbidden City Dim Sum & Seafood Restaurant', 63, 3.5)


In [72]:
# from SQL query back to dataframe
c.execute('''Select * from ccr order by review_count DESC limit 10''')
yelp_top10=pd.DataFrame(c.fetchall(), columns=['name', 'review_count', 'rating'])
print(yelp_top10)


                                          name  review_count  rating
0                           T.Pot China Bistro           112     3.5
1                     Silver Dragon Restaurant           111     3.5
2               Great Taste Chinese Restaurant            82     4.0
3                         Sun's BBQ Restaurant            79     3.5
4                     Calgary Court Restaurant            74     4.0
5                            U & Me Restaurant            73     3.5
6                     Ginger Beef Bistro House            69     3.5
7                          Bill’s Peking House            65     3.5
8                          Szechuan Restaurant            64     4.0
9  Forbidden City Dim Sum & Seafood Restaurant            63     3.5


## Part 2 Four Square API and SQL 
### step 1 FourSquar API Data Extraction

In [23]:
# Import libraries
import requests
import json
CLIENT_ID=os.environ['FOURSQUARE_CLIENT_ID']
CLIENT_SECRET=os.environ['FOURSQUARE_CLIENT_SECRET']
latitude='51.0430' #use calgary downtown lat & long for query
longitude='-114.0687'
VERSION=20200924
search_query='chinese restaurant'
radius=2000
limit=50
category_id='4bf58dd8d48988d145941735'

In [24]:
# get Foursqure API response for business name and id first
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, category_id, radius, limit)

results = requests.get(url).json()
info= results['response']['venues']

from pandas.io.json import json_normalize
df1=json_normalize(info)
df_fs=df1[['id','name']]
df_fs.head(2)

,id,name
0,4b0586ecf964a520a17522e3,Singapore Sam's
1,4fa8142de4b0bbcd4bd32a0e,Tiki-Ming
2,4b0586e7f964a520267422e3,Ho Won Restaurant
3,5c2af8cf7a0eff0025797a27,dagu rice noodle
4,5d3e077c56848800089d2f3d,Bubble Waffle Cafe
5,4b228f0df964a520fd4824e3,The Pink Pearl
6,53f3edcd498e02e3a48abac9,Auntie's Chinese Burger
7,4bca025f937ca59398dba692,Wok n Roll Bistro
8,4d631134855e5481c9f7f679,Red House Restaurant
9,4b293a9af964a520519b24e3,Kam Han Szechuan House


In [29]:
# convert the pandas data column to list
import pandas as pd
ids=list(set(df_fs['id']))
ids

['4ba904a5f964a52006043ae3',
 '4bb14fd5f964a5207d8b3ce3',
 '4b0586e7f964a520267422e3',
 '4f125973e4b09e81d9b56944',
 '4b37ac27f964a520114425e3',
 '4bac0431f964a52049de3ae3',
 '4b6b6ebef964a52070082ce3',
 '4d87a190e83fa143f975a8a1',
 '4b228f0df964a520fd4824e3',
 '58fea745aa6c9549ac21ef47',
 '5baec6dca2a6ce002ccd22c8',
 '53361ce8498e7258d12b5ce2',
 '522e13c311d265a2f92e4b86',
 '4b5f461bf964a52059b129e3',
 '4e2b52c8e4cd3bc166947b64',
 '4b3d4934f964a520a29125e3',
 '593303ecf427de60324216e4',
 '4bd0e9d6046076b0c8087171',
 '5c2af8cf7a0eff0025797a27',
 '4b0586ecf964a520a17522e3',
 '59407b3b81348810da371dfb',
 '52f82cca498ed0446247415a',
 '4b0586e7f964a520f77322e3',
 '53f3edcd498e02e3a48abac9',
 '4d6f0f18516b8cfa04374c10',
 '4b0586ebf964a520477522e3',
 '5c0f453b3ba767002c821b7a',
 '4fa8142de4b0bbcd4bd32a0e',
 '5247227311d28d1557679130',
 '4c4f3af579ffe21ec3494f1a',
 '4cae0f488c48a0933174712c',
 '4d22566e756e8cfa57cf7d54',
 '4d631134855e5481c9f7f679',
 '4d87a19002eb5481448a6ef5',
 '5d3e077c5684

In [35]:
# try one example to get the business detail for rating or likes
venue_id='4ba904a5f964a52006043ae3'
detail_url ='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
v1=requests.get(detail_url)
v1.json()['response']['venue'].keys()

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])

In [36]:
# extract the likes_count information and save in a list 
likes=[]
for id in ids:
    venue_id=id
    detail_url ='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    
    detail=requests.get(detail_url).json()
    like=detail['response']['venue']['likes']['count']
    likes.append(like)
 
print(likes)

[1, 0, 3, 16, 23, 0, 15, 0, 2, 0, 0, 0, 1, 1, 0, 9, 1, 0, 0, 11, 1, 0, 25, 1, 0, 13, 0, 0, 3, 4, 0, 6, 1, 0, 0, 0, 1, 6, 11, 10, 8, 3, 0, 3, 2, 1, 2]


In [48]:
df_fs['like_count']=likes
fs=df_fs[['name', 'like_count']]

In [49]:
fstop10=fs.sort_values(by='like_count', ascending=False).head(10)
fstop10

,name,like_count
22,Tang Dynasty,25
4,Bubble Waffle Cafe,23
3,dagu rice noodle,16
6,Auntie's Chinese Burger,15
25,Tiki-Ming Bankers Hall,13
19,成都川菜 Cheng Du Restaurant,11
38,Jerky King,11
39,Great Taste Chinese Restaurant,10
15,King's Restaurant,9
40,Regency Palace Restaurant,8


### step 2 Foursquare SQL

In [44]:
#convert back to SQL database
import sqlite3 as sqlite
conn1=sqlite.connect('q.db')
c=conn1.cursor()
c.execute('CREATE TABLE fs (name text, like_count integer)')
conn1.commit()

In [50]:
# Pandas to  SQL
fs.to_sql('fs', conn1, if_exists='replace', index=False)

In [51]:
#SQL Query 
c.execute('''Select * from fs order by like_count DESC limit 10''')
for row in c.fetchall():
    print(row)

('Tang Dynasty', 25)
('Bubble Waffle Cafe', 23)
('dagu rice noodle', 16)
("Auntie's Chinese Burger", 15)
('Tiki-Ming Bankers Hall', 13)
('成都川菜 Cheng Du Restaurant', 11)
('Jerky King', 11)
('Great Taste Chinese Restaurant', 10)
("King's Restaurant", 9)
('Regency Palace Restaurant', 8)


In [54]:
# print pandas data table from SQL query result
c.execute('''Select * from fs order by like_count DESC limit 10''')
fs_top10=pd.DataFrame(c.fetchall(), columns=['name', 'like_count'])
print(fs_top10)

                             name  like_count
0                    Tang Dynasty          25
1              Bubble Waffle Cafe          23
2                dagu rice noodle          16
3         Auntie's Chinese Burger          15
4          Tiki-Ming Bankers Hall          13
5        成都川菜 Cheng Du Restaurant          11
6                      Jerky King          11
7  Great Taste Chinese Restaurant          10
8               King's Restaurant           9
9       Regency Palace Restaurant           8


# final comparison of two API results


In [73]:
print(yelp_top10, fs_top10)

                                          name  review_count  rating
0                           T.Pot China Bistro           112     3.5
1                     Silver Dragon Restaurant           111     3.5
2               Great Taste Chinese Restaurant            82     4.0
3                         Sun's BBQ Restaurant            79     3.5
4                     Calgary Court Restaurant            74     4.0
5                            U & Me Restaurant            73     3.5
6                     Ginger Beef Bistro House            69     3.5
7                          Bill’s Peking House            65     3.5
8                          Szechuan Restaurant            64     4.0
9  Forbidden City Dim Sum & Seafood Restaurant            63     3.5                              name  like_count
0                    Tang Dynasty          25
1              Bubble Waffle Cafe          23
2                dagu rice noodle          16
3         Auntie's Chinese Burger          15
4          

## conclusion: -first choice: Yelp!
    1. Yelp API has both "rating" and "review_counts" and more user friendly, easy to get the response. 
    Foursquare needs more premiums to get the detailed rating and likes information.

    2. Yelp API provides more number of records for the ratings and review_counts, more popularized.
    
    3. Foursquare can get more detailed information reagrding the business (pictures, recommendation, tips et al)
    